In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
import datetime
import json
import folium
import plotly.express as px
from pprint import pprint
from bs4 import BeautifulSoup
import googlemaps
from time import sleep
from pandas_profiling import ProfileReport

#  Incendios en España en el período 2001 - 2015


In [2]:
df = pd.read_csv("fires-all 2001-2015.csv")

In [3]:
df

,id,superficie,fecha,lat,lng,latlng_explicit,idcomunidad,idprovincia,idmunicipio,municipio,...,causa_supuesta,causa_desc,muertos,heridos,time_ctrl,time_ext,personal,medios,gastos,perdidas
0,2001010001,3.70,2001-03-18,42.954656,-2.325719,1,1,1,13,BARRUNDIA,...,NaN,2,NaN,NaN,150,180,10,2,661.0,553.0
1,2001010004,1.50,2001-03-24,42.552183,-2.640673,1,1,1,41,NAVARIDAS,...,1.0,10,NaN,NaN,275,290,4,1,661.0,451.0
2,2001010005,1.50,2001-04-16,48.302507,-3.397798,1,1,1,33,LAPUEBLA DE LABARCA,...,NaN,10,NaN,NaN,135,165,4,1,312.0,902.0
3,2001010008,7.50,2001-05-25,42.946479,-2.485163,1,1,1,13,BARRUNDIA,...,NaN,2,NaN,NaN,335,410,6,1,1082.0,15338.0
4,2001010017,1.04,2001-07-20,43.091715,-3.024574,1,1,1,10,AIARA/AYALA,...,NaN,0,NaN,NaN,45,75,11,2,270.0,2866.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82635,2015500092,1.01,2015-07-10,41.324187,-1.791716,1,10,50,38,ATECA,...,NaN,20,NaN,NaN,63,198,26,3,NaN,1.0
82636,2015500116,11.20,2015-08-31,41.873662,-0.900144,1,10,50,298,ZUERA,...,NaN,0,NaN,NaN,110,455,49,5,NaN,21096.0
82637,2015500121,1.11,2015-09-15,41.214308,0.249973,1,10,50,189,NONASPE,...,NaN,10,NaN,NaN,69,329,15,2,NaN,1.0
82638,2015500126,1.28,2015-09-25,41.387613,-1.326090,1,10,50,24,ALMONACID DE LA SIERRA,...,NaN,0,NaN,NaN,51,211,19,4,NaN,1.0


###  Filtramos por las columnas que nos interesan

In [4]:
df_filtrado = df[['superficie', 'fecha', 'lat', 'lng','idprovincia', 'municipio', 'causa', 'time_ctrl',
       'time_ext', 'personal', 'medios', 'gastos', 'perdidas']]

In [5]:
df_filtrado

,superficie,fecha,lat,lng,idprovincia,municipio,causa,time_ctrl,time_ext,personal,medios,gastos,perdidas
0,3.70,2001-03-18,42.954656,-2.325719,1,BARRUNDIA,4,150,180,10,2,661.0,553.0
1,1.50,2001-03-24,42.552183,-2.640673,1,NAVARIDAS,2,275,290,4,1,661.0,451.0
2,1.50,2001-04-16,48.302507,-3.397798,1,LAPUEBLA DE LABARCA,2,135,165,4,1,312.0,902.0
3,7.50,2001-05-25,42.946479,-2.485163,1,BARRUNDIA,4,335,410,6,1,1082.0,15338.0
4,1.04,2001-07-20,43.091715,-3.024574,1,AIARA/AYALA,5,45,75,11,2,270.0,2866.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
82635,1.01,2015-07-10,41.324187,-1.791716,50,ATECA,3,63,198,26,3,NaN,1.0
82636,11.20,2015-08-31,41.873662,-0.900144,50,ZUERA,1,110,455,49,5,NaN,21096.0
82637,1.11,2015-09-15,41.214308,0.249973,50,NONASPE,3,69,329,15,2,NaN,1.0
82638,1.28,2015-09-25,41.387613,-1.326090,50,ALMONACID DE LA SIERRA,5,51,211,19,4,NaN,1.0


In [6]:
df1=df_filtrado. loc[df_filtrado["idprovincia"]==15]
df2=df_filtrado. loc[df_filtrado["idprovincia"]==36]
df3=df_filtrado. loc[df_filtrado["idprovincia"]==27]
df4=df_filtrado. loc[df_filtrado["idprovincia"]==32]

## Nos centramos solo en las provincias gallegas


In [7]:
df_final = pd.concat([df1, df2, df3, df4])
df_final

,superficie,fecha,lat,lng,idprovincia,municipio,causa,time_ctrl,time_ext,personal,medios,gastos,perdidas
1202,5.00,2001-02-20,43.703581,-8.038777,15,CEDEIRA,2,235,270,14,2,NaN,7013.0
1203,1.50,2001-02-24,42.936918,-9.114350,15,DUMBRÍA,4,470,530,14,1,NaN,1497.0
1204,3.00,2001-02-25,42.643031,-8.939252,15,"POBRA DO CARAMIÑAL, A",4,185,220,14,3,NaN,1882.0
1205,1.50,2001-02-25,43.186836,-8.685470,15,CARBALLO,4,125,135,5,0,NaN,1028.0
1206,3.80,2001-02-25,42.917476,-9.082862,15,MAZARICOS,4,1050,1051,14,1,NaN,3119.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80506,19.95,2015-12-19,42.230957,-7.197684,32,MANZANEDA,4,246,265,10,0,NaN,0.0
80507,37.70,2015-12-27,42.175632,-7.882516,32,"MERCA, A",2,864,1220,24,4,NaN,44698.0
80508,2.09,2015-12-27,42.261733,-6.766558,32,CARBALLEDA DE VALDEORRAS,5,460,622,13,0,NaN,0.0
80509,1.50,2015-12-27,42.545012,-8.142964,32,"IRIXO, O",4,186,202,4,0,NaN,0.0


In [8]:
df5=df_final. loc[df_final["causa"]==5]

In [9]:
df6=df_final. loc[df_final["causa"]==6]

### Seleccionamos solo los incendios que no fueron intencionados

In [10]:
df_incendios=pd.concat([df5,df6])
df_incendios

,superficie,fecha,lat,lng,idprovincia,municipio,causa,time_ctrl,time_ext,personal,medios,gastos,perdidas
1208,1.10,2001-04-13,42.758649,-8.917814,15,LOUSAME,5,190,210,18,2,NaN,234.0
1217,1.70,2001-04-17,43.291593,-8.495562,15,ARTEIXO,5,200,245,9,1,NaN,2175.0
1218,1.40,2001-04-17,43.307140,-8.463079,15,ARTEIXO,5,125,170,16,2,NaN,541.0
1253,4.50,2001-05-19,43.204208,-8.088987,15,ARANGA,5,320,550,24,1,NaN,5030.0
1260,3.00,2001-05-22,43.297494,-8.395937,15,CULLEREDO,5,285,325,24,2,NaN,2692.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80394,2.96,2015-08-12,42.225960,-7.576174,32,MACEDA,6,177,334,31,11,NaN,7447.0
80413,59.59,2015-08-22,42.372595,-6.801253,32,CARBALLEDA DE VALDEORRAS,6,360,385,64,13,NaN,0.0
80449,20.07,2015-09-04,42.247517,-7.214861,32,MANZANEDA,6,328,440,41,3,NaN,0.0
80452,26.44,2015-09-04,42.365928,-7.871268,32,OURENSE,6,1044,1497,217,28,NaN,279.0


In [11]:
df_incendios['municipio'] = df_incendios['municipio'].str.lower()

#### Tenemos que eliminar 5 filas porque no defnían en que provincia habían sido

In [12]:
df_incendios = df_incendios.drop(63543)
df_incendios = df_incendios.drop(69200)
df_incendios = df_incendios.drop(69232)
df_incendios = df_incendios.drop(68504)
df_incendios = df_incendios.drop(53852)

In [13]:
df_incendios

,superficie,fecha,lat,lng,idprovincia,municipio,causa,time_ctrl,time_ext,personal,medios,gastos,perdidas
1208,1.10,2001-04-13,42.758649,-8.917814,15,lousame,5,190,210,18,2,NaN,234.0
1217,1.70,2001-04-17,43.291593,-8.495562,15,arteixo,5,200,245,9,1,NaN,2175.0
1218,1.40,2001-04-17,43.307140,-8.463079,15,arteixo,5,125,170,16,2,NaN,541.0
1253,4.50,2001-05-19,43.204208,-8.088987,15,aranga,5,320,550,24,1,NaN,5030.0
1260,3.00,2001-05-22,43.297494,-8.395937,15,culleredo,5,285,325,24,2,NaN,2692.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
80394,2.96,2015-08-12,42.225960,-7.576174,32,maceda,6,177,334,31,11,NaN,7447.0
80413,59.59,2015-08-22,42.372595,-6.801253,32,carballeda de valdeorras,6,360,385,64,13,NaN,0.0
80449,20.07,2015-09-04,42.247517,-7.214861,32,manzaneda,6,328,440,41,3,NaN,0.0
80452,26.44,2015-09-04,42.365928,-7.871268,32,ourense,6,1044,1497,217,28,NaN,279.0


###  Tabla de todos los municipios con su correspondiente estación meteorológica

In [14]:
df_ema = pd.read_excel("ema.xlsx")
df_ema

,idprovincia,municipio,ema,Latitud,Longitud,Identificador,num_ema
0,15,Abegondo,A CORUÑA,432157N,082517W,1387,1
1,15,Ares,A CORUÑA,432157N,082517W,1387,1
2,15,Bergondo,A CORUÑA,432157N,082517W,1387,1
3,15,Betanzos,A CORUÑA,432157N,082517W,1387,1
4,15,Cabanas,A CORUÑA,432157N,082517W,1387,1
...,...,...,...,...,...,...,...
310,32,Vilar de Barrio,XINZO DE LIMIA,420445N,074401W,1735X,13
311,32,Vilar de Santos,XINZO DE LIMIA,420445N,074401W,1735X,13
312,32,Vilardevós,XINZO DE LIMIA,420445N,074401W,1735X,13
313,32,Xinzo de Limia,XINZO DE LIMIA,420445N,074401W,1735X,13


In [15]:
df_ema['municipio'] = df_ema['municipio'].str.lower()

### Para asignar cada incendio a su estación meteorológica debemos hacer limpieza de datos

In [16]:
df_ema.loc[df_ema['municipio'] == 'carballiño, o', 'municipio'] = 'carballino, o'
df_ema.loc[df_ema['municipio'] == 'muíños', 'municipio'] = 'muiños'
df_ema.loc[df_ema['municipio'] == 'pobra do brollón, a', 'municipio'] = 'pobla do brollón, a'
df_ema.loc[df_ema['municipio'] == 'pontes de garcía rodríguez, as', 'municipio'] = 'pontes de garcia rodriguez, as'
df_ema.loc[df_ema['municipio'] == 'san xoán de río', 'municipio'] = 'san xoan de rio'


In [17]:
lista_vacia=[]
id_vacia=[]
elementos_def = df_incendios['municipio'].tolist()
elementos_ema_mun = df_ema['municipio'].tolist()
elementos_ema_ema = df_ema['ema'].tolist()
elementos_ema_id = df_ema['Identificador'].tolist()

print(len(elementos_ema_ema))

315


In [18]:
for i in range(len(elementos_def)):
    for j in range (len(elementos_ema_mun)):
        if elementos_def[i] == elementos_ema_mun[j]:
            lista_vacia.append(elementos_ema_ema[j])
            id_vacia.append(elementos_ema_id[j])
print(len(lista_vacia))

2145


In [19]:
nueva_lista_def = sorted(elementos_def)
nueva_lista_ema = sorted(elementos_ema_mun)


In [20]:
df_incendios = df_incendios.assign(ema=lista_vacia)
df_incendios = df_incendios.assign(id_ema=id_vacia)

## Tabla final sobre la que trabajar

In [21]:
df_incendios

,superficie,fecha,lat,lng,idprovincia,municipio,causa,time_ctrl,time_ext,personal,medios,gastos,perdidas,ema,id_ema
1208,1.10,2001-04-13,42.758649,-8.917814,15,lousame,5,190,210,18,2,NaN,234.0,MONTE IROITE,1437O
1217,1.70,2001-04-17,43.291593,-8.495562,15,arteixo,5,200,245,9,1,NaN,2175.0,A CORUÑA AEROPUERTO,1387E
1218,1.40,2001-04-17,43.307140,-8.463079,15,arteixo,5,125,170,16,2,NaN,541.0,A CORUÑA AEROPUERTO,1387E
1253,4.50,2001-05-19,43.204208,-8.088987,15,aranga,5,320,550,24,1,NaN,5030.0,SANTIAGO AEROPUERTO,1428
1260,3.00,2001-05-22,43.297494,-8.395937,15,culleredo,5,285,325,24,2,NaN,2692.0,A CORUÑA AEROPUERTO,1387E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80394,2.96,2015-08-12,42.225960,-7.576174,32,maceda,6,177,334,31,11,NaN,7447.0,OURENSE,1690A
80413,59.59,2015-08-22,42.372595,-6.801253,32,carballeda de valdeorras,6,360,385,64,13,NaN,0.0,A POBRA DE TRIVES,1631E
80449,20.07,2015-09-04,42.247517,-7.214861,32,manzaneda,6,328,440,41,3,NaN,0.0,A POBRA DE TRIVES,1631E
80452,26.44,2015-09-04,42.365928,-7.871268,32,ourense,6,1044,1497,217,28,NaN,279.0,OURENSE,1690A


In [22]:
anos=[]
meses=[]
dias=[]

fecha = df_incendios['fecha'].tolist()
for i in range (2145):
    anos.append(fecha[i][0:4])
    meses.append(fecha[i][5:7])
    dias.append(fecha[i][8:10])
    
    


In [23]:
df_incendios = df_incendios.assign(año=anos)
df_incendios = df_incendios.assign(mes=meses)
df_incendios = df_incendios.assign(dia=dias)

In [24]:
df_incendios["año"] = df_incendios["año"].astype(float)
df_incendios["mes"] = df_incendios["mes"].astype(float)
df_incendios["dia"] = df_incendios["dia"].astype(float)


In [25]:
df_incendios

,superficie,fecha,lat,lng,idprovincia,municipio,causa,time_ctrl,time_ext,personal,medios,gastos,perdidas,ema,id_ema,año,mes,dia
1208,1.10,2001-04-13,42.758649,-8.917814,15,lousame,5,190,210,18,2,NaN,234.0,MONTE IROITE,1437O,2001.0,4.0,13.0
1217,1.70,2001-04-17,43.291593,-8.495562,15,arteixo,5,200,245,9,1,NaN,2175.0,A CORUÑA AEROPUERTO,1387E,2001.0,4.0,17.0
1218,1.40,2001-04-17,43.307140,-8.463079,15,arteixo,5,125,170,16,2,NaN,541.0,A CORUÑA AEROPUERTO,1387E,2001.0,4.0,17.0
1253,4.50,2001-05-19,43.204208,-8.088987,15,aranga,5,320,550,24,1,NaN,5030.0,SANTIAGO AEROPUERTO,1428,2001.0,5.0,19.0
1260,3.00,2001-05-22,43.297494,-8.395937,15,culleredo,5,285,325,24,2,NaN,2692.0,A CORUÑA AEROPUERTO,1387E,2001.0,5.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80394,2.96,2015-08-12,42.225960,-7.576174,32,maceda,6,177,334,31,11,NaN,7447.0,OURENSE,1690A,2015.0,8.0,12.0
80413,59.59,2015-08-22,42.372595,-6.801253,32,carballeda de valdeorras,6,360,385,64,13,NaN,0.0,A POBRA DE TRIVES,1631E,2015.0,8.0,22.0
80449,20.07,2015-09-04,42.247517,-7.214861,32,manzaneda,6,328,440,41,3,NaN,0.0,A POBRA DE TRIVES,1631E,2015.0,9.0,4.0
80452,26.44,2015-09-04,42.365928,-7.871268,32,ourense,6,1044,1497,217,28,NaN,279.0,OURENSE,1690A,2015.0,9.0,4.0


In [26]:
df_incendios = df_incendios.rename(columns={'id_ema': 'indicativo'})
df_incendios

,superficie,fecha,lat,lng,idprovincia,municipio,causa,time_ctrl,time_ext,personal,medios,gastos,perdidas,ema,indicativo,año,mes,dia
1208,1.10,2001-04-13,42.758649,-8.917814,15,lousame,5,190,210,18,2,NaN,234.0,MONTE IROITE,1437O,2001.0,4.0,13.0
1217,1.70,2001-04-17,43.291593,-8.495562,15,arteixo,5,200,245,9,1,NaN,2175.0,A CORUÑA AEROPUERTO,1387E,2001.0,4.0,17.0
1218,1.40,2001-04-17,43.307140,-8.463079,15,arteixo,5,125,170,16,2,NaN,541.0,A CORUÑA AEROPUERTO,1387E,2001.0,4.0,17.0
1253,4.50,2001-05-19,43.204208,-8.088987,15,aranga,5,320,550,24,1,NaN,5030.0,SANTIAGO AEROPUERTO,1428,2001.0,5.0,19.0
1260,3.00,2001-05-22,43.297494,-8.395937,15,culleredo,5,285,325,24,2,NaN,2692.0,A CORUÑA AEROPUERTO,1387E,2001.0,5.0,22.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80394,2.96,2015-08-12,42.225960,-7.576174,32,maceda,6,177,334,31,11,NaN,7447.0,OURENSE,1690A,2015.0,8.0,12.0
80413,59.59,2015-08-22,42.372595,-6.801253,32,carballeda de valdeorras,6,360,385,64,13,NaN,0.0,A POBRA DE TRIVES,1631E,2015.0,8.0,22.0
80449,20.07,2015-09-04,42.247517,-7.214861,32,manzaneda,6,328,440,41,3,NaN,0.0,A POBRA DE TRIVES,1631E,2015.0,9.0,4.0
80452,26.44,2015-09-04,42.365928,-7.871268,32,ourense,6,1044,1497,217,28,NaN,279.0,OURENSE,1690A,2015.0,9.0,4.0


In [27]:
df_proximos = pd.read_excel("cercanos_emas.xlsx")
df_proximos

,Municipio
0,Ames
1,Amoeiro
2,Arteixo
3,Barbadas
4,Barro
...,...
89,Vilaboa
90,Vilamarín
91,Vilar de Santos
92,Vimianzo


In [28]:
df_proximos['Municipio'] = df_proximos['Municipio'].str.lower()

In [29]:
listado = df_proximos['Municipio'].tolist()


In [30]:
df_copia = df_incendios.copy()

In [31]:
df_copia[df_copia["municipio"].isin(listado)]

,superficie,fecha,lat,lng,idprovincia,municipio,causa,time_ctrl,time_ext,personal,medios,gastos,perdidas,ema,indicativo,año,mes,dia
1208,1.10,2001-04-13,42.758649,-8.917814,15,lousame,5,190,210,18,2,NaN,234.0,MONTE IROITE,1437O,2001.0,4.0,13.0
1217,1.70,2001-04-17,43.291593,-8.495562,15,arteixo,5,200,245,9,1,NaN,2175.0,A CORUÑA AEROPUERTO,1387E,2001.0,4.0,17.0
1218,1.40,2001-04-17,43.307140,-8.463079,15,arteixo,5,125,170,16,2,NaN,541.0,A CORUÑA AEROPUERTO,1387E,2001.0,4.0,17.0
1330,1.00,2001-06-24,42.763280,-8.743060,15,rois,5,70,190,33,1,NaN,468.0,PADRÓN,1473A,2001.0,6.0,24.0
1331,2.50,2001-06-24,42.797346,-8.681299,15,rois,5,155,190,2,2,NaN,1700.0,PADRÓN,1473A,2001.0,6.0,24.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73847,4.00,2013-08-28,42.522358,-7.907439,32,san cristovo de cea,6,169,446,44,17,NaN,4313.0,O CARBALLIÑO,1700X,2013.0,8.0,28.0
73883,1.00,2013-09-05,42.474599,-7.882954,32,vilamarín,6,90,265,11,2,NaN,6600.0,OURENSE,1690A,2013.0,9.0,5.0
80285,1.20,2015-07-08,42.390335,-7.860431,32,coles,6,80,200,8,3,NaN,0.0,OURENSE,1690A,2015.0,7.0,8.0
80449,20.07,2015-09-04,42.247517,-7.214861,32,manzaneda,6,328,440,41,3,NaN,0.0,A POBRA DE TRIVES,1631E,2015.0,9.0,4.0


In [33]:
df_datos_totales = pd.read_csv("df_datos_est.csv")

In [34]:
df_datos_totales

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,tmax,dir,...,Month,hr,tm_min,nt_30,w_racha,np_100,ta_min,e,Day_y,ID
0,2001-01-01,1387E,A CORUÑA AEROPUERTO,A CORUÑA,98,12.0,11.7,9.8,14.1,25.0,...,1,82.0,7.3,0.0,33.0,9.0,1.8,105.0,1,0
1,2001-01-02,1387E,A CORUÑA AEROPUERTO,A CORUÑA,98,9.8,7.2,8.0,11.5,33.0,...,1,82.0,7.3,0.0,33.0,9.0,1.8,105.0,1,0
2,2001-01-03,1387E,A CORUÑA AEROPUERTO,A CORUÑA,98,11.8,30.1,9.2,14.4,29.0,...,1,82.0,7.3,0.0,33.0,9.0,1.8,105.0,1,0
3,2001-01-04,1387E,A CORUÑA AEROPUERTO,A CORUÑA,98,12.4,31.8,10.1,14.6,23.0,...,1,82.0,7.3,0.0,33.0,9.0,1.8,105.0,1,0
4,2001-01-05,1387E,A CORUÑA AEROPUERTO,A CORUÑA,98,12.4,16.7,8.3,16.4,28.0,...,1,82.0,7.3,0.0,33.0,9.0,1.8,105.0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76088,2014-12-27,1495,VIGO AEROPUERTO,PONTEVEDRA,255,8.6,1.8,4.1,13.2,26.0,...,12,85.0,5.6,0.0,2.0,1.0,0.7,99.0,1,15
76089,2014-12-28,1495,VIGO AEROPUERTO,PONTEVEDRA,255,8.5,0.9,4.8,12.2,35.0,...,12,85.0,5.6,0.0,2.0,1.0,0.7,99.0,1,15
76090,2014-12-29,1495,VIGO AEROPUERTO,PONTEVEDRA,255,8.0,0.0,2.2,13.8,5.0,...,12,85.0,5.6,0.0,2.0,1.0,0.7,99.0,1,15
76091,2014-12-30,1495,VIGO AEROPUERTO,PONTEVEDRA,255,6.6,0.0,0.7,12.5,9.0,...,12,85.0,5.6,0.0,2.0,1.0,0.7,99.0,1,15


In [35]:
df_result = pd.merge(df_incendios, df_datos_totales, on=['fecha', 'indicativo'])
df_result

,superficie,fecha,lat,lng,idprovincia,municipio,causa,time_ctrl,time_ext,personal,...,Month,hr,tm_min,nt_30,w_racha,np_100,ta_min,e,Day_y,ID
0,1.10,2001-04-13,42.758649,-8.917814,15,lousame,5,190,210,18,...,4,73.0,4.8,0.0,NaN,6.0,1.6,NaN,1,5
1,1.70,2001-04-17,43.291593,-8.495562,15,arteixo,5,200,245,9,...,4,71.0,7.9,0.0,20.0,0.0,3.0,104.0,1,0
2,1.40,2001-04-17,43.307140,-8.463079,15,arteixo,5,125,170,16,...,4,71.0,7.9,0.0,20.0,0.0,3.0,104.0,1,0
3,3.00,2001-05-22,43.297494,-8.395937,15,culleredo,5,285,325,24,...,5,73.0,9.8,0.0,6.0,2.0,4.5,125.0,1,0
4,1.00,2001-06-24,42.763280,-8.743060,15,rois,5,70,190,33,...,6,62.0,12.5,4.0,7.0,0.0,8.0,136.0,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,2.36,2013-08-28,42.316546,-7.555640,32,montederramo,6,120,597,30,...,8,55.0,11.2,4.0,24.0,0.0,7.0,128.0,1,10
1275,4.00,2013-08-28,42.522358,-7.907439,32,san cristovo de cea,6,169,446,44,...,8,63.0,11.9,23.0,NaN,0.0,8.4,167.0,1,11
1276,1.00,2013-09-05,42.474599,-7.882954,32,vilamarín,6,90,265,11,...,9,60.0,13.4,15.0,28.0,3.0,6.7,148.0,1,12
1277,1.50,2013-09-12,41.988375,-6.995473,32,"mezquita, a",6,110,172,16,...,9,64.0,11.2,8.0,31.0,2.0,6.0,139.0,1,13


### Visualizamos los incendios de mas de 150 hectáreas

In [37]:
map= folium.Map(location=[df_result.iloc[0]['lat'], df_result.iloc[0]['lng']], 
                  tiles='OpenStreetMap',
                  zoom_start = 11)
df_result

,superficie,fecha,lat,lng,idprovincia,municipio,causa,time_ctrl,time_ext,personal,...,Month,hr,tm_min,nt_30,w_racha,np_100,ta_min,e,Day_y,ID
0,1.10,2001-04-13,42.758649,-8.917814,15,lousame,5,190,210,18,...,4,73.0,4.8,0.0,NaN,6.0,1.6,NaN,1,5
1,1.70,2001-04-17,43.291593,-8.495562,15,arteixo,5,200,245,9,...,4,71.0,7.9,0.0,20.0,0.0,3.0,104.0,1,0
2,1.40,2001-04-17,43.307140,-8.463079,15,arteixo,5,125,170,16,...,4,71.0,7.9,0.0,20.0,0.0,3.0,104.0,1,0
3,3.00,2001-05-22,43.297494,-8.395937,15,culleredo,5,285,325,24,...,5,73.0,9.8,0.0,6.0,2.0,4.5,125.0,1,0
4,1.00,2001-06-24,42.763280,-8.743060,15,rois,5,70,190,33,...,6,62.0,12.5,4.0,7.0,0.0,8.0,136.0,1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1274,2.36,2013-08-28,42.316546,-7.555640,32,montederramo,6,120,597,30,...,8,55.0,11.2,4.0,24.0,0.0,7.0,128.0,1,10
1275,4.00,2013-08-28,42.522358,-7.907439,32,san cristovo de cea,6,169,446,44,...,8,63.0,11.9,23.0,NaN,0.0,8.4,167.0,1,11
1276,1.00,2013-09-05,42.474599,-7.882954,32,vilamarín,6,90,265,11,...,9,60.0,13.4,15.0,28.0,3.0,6.7,148.0,1,12
1277,1.50,2013-09-12,41.988375,-6.995473,32,"mezquita, a",6,110,172,16,...,9,64.0,11.2,8.0,31.0,2.0,6.0,139.0,1,13


In [38]:

mapa = folium.Map(location = [df_result.iloc[0]['lat'], df_result.iloc[0]['lng']], zoom_start = 8)



In [39]:
df_inc_grande=df_result. loc[df_result["superficie"]>150]
df_inc_grande

,superficie,fecha,lat,lng,idprovincia,municipio,causa,time_ctrl,time_ext,personal,...,Month,hr,tm_min,nt_30,w_racha,np_100,ta_min,e,Day_y,ID
122,471.00,2006-08-05,42.739458,-8.822374,15,lousame,5,1830,2075,79,...,8,78.0,12.9,0.0,NaN,NaN,8.6,NaN,1,5
126,477.75,2006-08-08,42.777547,-9.075976,15,muros,5,455,1725,45,...,8,78.0,12.9,0.0,NaN,NaN,8.6,NaN,1,5
146,222.00,2006-09-05,42.821208,-8.714818,15,rois,5,710,1670,107,...,9,69.0,14.2,6.0,99.0,4.0,7.9,152.0,1,6
258,1544.90,2006-08-06,42.528964,-8.478438,36,campo lameiro,6,960,2100,31,...,8,61.0,15.8,13.0,36.0,3.0,11.6,165.0,1,14
268,834.30,2006-08-12,42.401757,-8.332789,36,"lama, a",5,1515,4275,23,...,8,61.0,15.8,13.0,36.0,3.0,11.6,165.0,1,14
495,195.00,2003-03-16,42.467646,-7.085108,32,vilamartín de valdeorras,5,1200,1420,49,...,3,80.0,3.3,0.0,NaN,NaN,NaN,NaN,1,10
512,280.00,2003-08-05,41.968512,-7.388564,32,verín,5,1350,1680,108,...,8,68.0,9.7,0.0,NaN,NaN,NaN,NaN,1,13
515,230.00,2003-08-12,42.294590,-7.712152,32,paderne de allariz,5,1075,1185,79,...,8,55.0,17.3,23.0,20.0,1.0,11.8,172.0,1,12
631,610.00,2005-08-20,42.126351,-7.949477,32,san cibrao das viñas,5,1440,1620,34,...,8,55.0,14.6,21.0,NaN,0.0,10.4,166.0,1,12
632,600.00,2005-08-20,42.126351,-7.949477,32,"merca, a",5,2980,3325,110,...,8,55.0,14.6,21.0,NaN,0.0,10.4,166.0,1,12


In [43]:
# Inicializa el mapa
sf_map = folium.Map(location = [42.52896, -8.47843], zoom_start = 7)


sf_map

In [42]:
 
incendios = folium.map.FeatureGroup()




for lat, lng, muni, superf in zip(df_inc_grande["lat"], df_inc_grande["lng"], df_inc_grande["municipio"], df_inc_grande["superficie"]):
    incendios.add_child(folium.Marker(location = [lat, lng],
                                      popup          = [muni, f"Superficie quemada: {superf}"],
                                            icon     = folium.Icon(icon = "neighborhood",
                                            icon_color     = "white",                       
                                            color          = "darkred",
                                            prefix         = "fa")))
    



sf_map.add_child(incendios)
sf_map



